In [1]:
import pandas as pd
import itertools
import os
import pandas.io.formats.style

In [2]:
pd.options.display.float_format = "{:,.3f}".format

### Globals, Helper Functions, etc

In [3]:
MEASURE_NAME_DICT = {
    "AlignedCosineSimilarity": "AlignCos",
    "CKA": "CKA",
    "ConcentricityDifference": "ConcDiff",
    "DistanceCorrelation": "DistCorr",
    "EigenspaceOverlapScore": "EOS",
    "GeometryScore": "GS",
    "Gulp": "GULP",
    "HardCorrelationMatch": "HardCorr",
    "IMDScore": "IMD",
    "JaccardSimilarity": "Jaccard",
    "LinearRegression": "LinReg",
    "MagnitudeDifference": "MagDiff",
    "OrthogonalAngularShapeMetricCentered": "ShapeMet",
    "OrthogonalProcrustesCenteredAndNormalized": "OrthProc",
    "PWCCA": "PWCCA",
    "PermutationProcrustes": "PermProc",
    "ProcrustesSizeAndShapeDistance": "ProcDist",
    "RSA": "RSA",
    "RSMNormDifference": "RSMDiff",
    "RankSimilarity": "RankSim",
    "SVCCA": "SVCCA",
    "SecondOrderCosineSimilarity": "2nd-Cos",
    "SoftCorrelationMatch": "SoftCorr",
    "UniformityDifference": "UnifDiff",
}

PIVOT_COL_DICT = {
    "cora": "Cora",
    "flickr": "Flickr",
    "ogbn-arxiv": "OGBN-Arxiv",
    "GraphSAGE": "SAGE",
    "violation_rate": "Violation Rate",
    "correlation": "Spearman Correlation"
}

COLUMN_NAME_DICT = {
    "similarity_measure": "Similarity Measure", 
    "quality_measure": "Measure",
    "functional_similarity_measure": "Measure",
    "architecture": "Model",
    "representation_dataset": "Dataset"
}

LATEX_FORMAT_DICT = {
    "hrules": True,
    "column_format": "l||rrr|rrr|rrr||rrr|rrr|rrr",
    "multicol_align": "c",
}

In [4]:
EXPERIMENT_RESULTS_PATH = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "experiments", "results", "full_csvs")
COMPARISON_TYPE_DICT = {
    "label_test": "group_separation",
    "layer_test": "monotonicity",
    "augmentation_test": "group_separation",
    "shortcut_test": "group_separation"
}

EXPERIMENTS = list(COMPARISON_TYPE_DICT.keys())
DATASETS = ["cora", "flickr", "ogbn-arxiv"]
ARCHITECTURES = ["GCN", "GraphSAGE", "GAT"]

def FULL_DF_FILE_NAME(experiment, comparison_type, dataset):
    return f"graphs_{experiment}_{dataset}_full.csv"

In [5]:
def get_pivot_table(experiment, dataset):
    path = os.path.join(EXPERIMENT_RESULTS_PATH, FULL_DF_FILE_NAME(experiment, COMPARISON_TYPE_DICT[experiment], dataset))
    
    df = pd.read_csv(path)
    data = df.loc[:, ["similarity_measure", "quality_measure", "value", "architecture", "representation_dataset"]]
    return data.pivot(index="similarity_measure", columns=["representation_dataset", "architecture", "quality_measure"], values="value")

In [6]:
def get_agg_pivot_table(experiment, datasets):
    dfs = []
    for dataset in datasets:
        path = os.path.join(EXPERIMENT_RESULTS_PATH, FULL_DF_FILE_NAME(experiment, COMPARISON_TYPE_DICT[experiment], dataset))
        df = pd.read_csv(path)
        data = df.loc[:, ["similarity_measure", "quality_measure", "value", "architecture", "representation_dataset"]].dropna()
   
        data = data.rename(COLUMN_NAME_DICT, axis="columns")
        dfs.append(data.iloc[:])
    df_cc = pd.concat(dfs, axis=0)
    df_res = df_cc.pivot_table(index="Similarity Measure", columns=["Measure", "Dataset", "Model"], values="value", aggfunc="mean")
    df_res = df_res.rename(MEASURE_NAME_DICT, axis="index")
    return df_res.rename(PIVOT_COL_DICT, axis="columns")

### Results for Group Separation Experiments

#### Random Label Test Results

In [7]:
get_agg_pivot_table("label_test", ["cora", "flickr", "ogbn-arxiv"])

Measure            AUPRC                                                  \
Dataset             Cora                   Flickr             OGBN-Arxiv   
Model                GAT   GCN  SAGE  PGNN    GAT   GCN  SAGE        GAT   
Similarity Measure                                                         
AlignCos           0.433 0.457 0.479 0.233  0.286 0.839 0.423      0.931   
CKA                0.424 0.427 0.424 0.247  0.270 0.733 0.657      1.000   
ConcDiff           0.394 0.203 0.391 0.187  0.208 0.366 0.570      0.997   
DistCorr           0.424 0.431 0.444 0.255  0.217 0.860 0.427      1.000   
EOS                0.284 0.295 0.259 0.245  0.219 0.413 0.424      0.340   
GULP               0.285 0.276 0.260 0.263  0.227 0.192 0.424      0.303   
HardCorr           0.424 0.425 0.424 0.226  0.332 0.771 0.457      0.544   
IMD                0.839 0.729 0.978 0.239  0.228 0.236 0.369      0.898   
Jaccard            0.427 0.425 0.424 0.258  0.292 0.564 0.430      0.778   
LinReg             0.456 0.492 0.431 0.235  0.230 0.223 0.449      0.467   
MagDiff            0.370 0.199 0.243 0.282  0.185 0.661 0.718      0.343   
ShapeMet           0.424 0.426 0.424 0.236  0.275 0.882 0.434      0.767   
OrthProc           0.424 0.426 0.424 0.236  0.275 0.882 0.434      0.767   
PWCCA              0.277 0.265 0.325 0.244  0.243   NaN 0.444      0.362   
PermProc           0.435 0.443 0.386 0.284  0.192 0.772 0.904      0.931   
ProcDist           0.433 0.452 0.451 0.284  0.190 0.785 0.625      1.000   
RSA                0.425 0.457 0.435 0.241  0.334 0.737 0.424      0.490   
RSMDiff            0.779 0.518 0.527 0.299  0.189 0.715 0.915      1.000   
RankSim            0.475 0.452 0.423 0.208  0.333 0.470 0.426      0.779   
SVCCA              0.422 0.347 0.306 0.194  0.268 0.332 0.796      0.932   
2nd-Cos            0.733 0.559 0.735 0.224  0.365 0.611 0.423      0.950   
SoftCorr           0.423 0.425 0.424 0.233  0.332 0.603 0.449      0.547   
UnifDiff           0.320 0.334 0.497 0.247  0.396 0.531 0.902      0.334   

Measure                        Violation Rate                                 \
Dataset                                  Cora                   Flickr         
Model                GCN  SAGE            GAT   GCN  SAGE  PGNN    GAT   GCN   
Similarity Measure                                                             
AlignCos           0.981 0.699          0.399 0.341 0.336 0.448  0.421 0.064   
CKA                1.000 1.000          0.493 0.451 0.488 0.431  0.458 0.093   
ConcDiff           0.964 1.000          0.261 0.455 0.219 0.570  0.519 0.277   
DistCorr           1.000 1.000          0.479 0.410 0.369 0.356  0.482 0.052   
EOS                0.253 0.426          0.342 0.351 0.449 0.438  0.419 0.500   
GULP               0.302 0.426          0.341 0.369 0.444 0.499  0.429 0.497   
HardCorr           0.828 0.828          0.492 0.473 0.489 0.474  0.333 0.059   
IMD                1.000 1.000          0.039 0.071 0.006 0.469  0.461 0.531   
Jaccard            0.827 0.432          0.447 0.473 0.492 0.431  0.432 0.229   
LinReg             0.444 0.677          0.312 0.287 0.421 0.490  0.484 0.511   
MagDiff            0.551 0.490          0.263 0.488 0.396 0.386  0.528 0.108   
ShapeMet           0.826 0.726          0.478 0.461 0.487 0.416  0.467 0.048   
OrthProc           0.826 0.726          0.478 0.461 0.487 0.416  0.467 0.048   
PWCCA              0.241 0.436          0.356 0.399 0.482 0.469  0.466   NaN   
PermProc           0.678 0.724          0.391 0.355 0.286 0.512  0.500 0.061   
ProcDist           0.915 0.983          0.403 0.327 0.323 0.518  0.490 0.068   
RSA                0.961 0.430          0.461 0.307 0.385 0.437  0.363 0.115   
RSMDiff            0.997 1.000          0.137 0.268 0.169 0.390  0.491 0.089   
RankSim            0.844 0.552          0.357 0.345 0.465 0.525  0.417 0.337   
SVCCA              1.000 1.000          0.409 0.381 0.445 0.617  0.457 0.311   
2nd-Cos            0.994 0.

In [8]:
label_test_df = get_agg_pivot_table("label_test", ["cora", "flickr", "ogbn-arxiv"])
styled = pd.io.formats.style.Styler(
    label_test_df,
    precision=2,
)

latex_str = styled.highlight_max(axis=0, props="textbf:--rwrap;").to_latex("graphs_results_label_test.tex", **LATEX_FORMAT_DICT)

#### Shortcut Test Results

In [9]:
get_agg_pivot_table("shortcut_test", ["cora", "flickr", "ogbn-arxiv"])

Measure            AUPRC                                                  \
Dataset             Cora                   Flickr             OGBN-Arxiv   
Model                GAT   GCN  SAGE  PGNN    GAT   GCN  SAGE        GAT   
Similarity Measure                                                         
AlignCos           0.588 0.635 0.599 0.434  0.480 0.895 1.000      1.000   
CKA                0.784 0.614 0.777 0.341  0.330 0.282 1.000      1.000   
ConcDiff           0.204 0.512 0.169 0.176  0.324 0.182 0.176      1.000   
DistCorr           0.802 0.661 0.820 0.420  0.317 0.333 1.000      1.000   
EOS                0.197 0.301 0.495 0.377  0.544 0.461 0.433      0.968   
GULP               0.197 0.310 0.496 0.372  0.451 0.196 0.433      0.965   
HardCorr           0.294 0.277 0.347 0.351  0.519 0.549 1.000      0.835   
IMD                0.589 0.699 0.814 0.263  0.357 0.783 0.971      0.916   
Jaccard            0.864 0.736 0.782 0.418  0.545 1.000 0.828      1.000   
LinReg             0.502 0.699 0.742 0.340  0.357 0.365 0.609      1.000   
MagDiff            0.181 0.366 0.151 0.215  0.198 0.266 0.783      1.000   
ShapeMet           0.619 0.629 0.794 0.347  0.307 0.581 1.000      1.000   
OrthProc           0.619 0.629 0.794 0.347  0.307 0.581 1.000      1.000   
PWCCA              0.215 0.328 0.468 0.382  0.329   NaN 0.433      0.997   
PermProc           0.246 0.230 0.268 0.251  0.272 0.418 1.000      0.766   
ProcDist           0.676 0.603 0.812 0.247  0.331 0.599 1.000      1.000   
RSA                0.726 0.490 0.432 0.420  0.521 0.894 1.000      0.902   
RSMDiff            0.689 0.462 0.497 0.368  0.362 0.476 0.920      1.000   
RankSim            0.849 0.732 0.645 0.426  0.538 1.000 0.774      1.000   
SVCCA              0.458 0.232 0.357 0.239  0.315 0.239 0.929      0.825   
2nd-Cos            0.807 0.846 0.907 0.335  0.522 1.000 1.000      1.000   
SoftCorr           0.302 0.306 0.404 0.391  0.571 0.663 1.000      0.830   
UnifDiff           0.712 0.296 0.637 0.298  0.480 0.593 0.499      0.974   

Measure                        Violation Rate                                 \
Dataset                                  Cora                   Flickr         
Model                GCN  SAGE            GAT   GCN  SAGE  PGNN    GAT   GCN   
Similarity Measure                                                             
AlignCos           1.000 1.000          0.107 0.160 0.119 0.335  0.231 0.019   
CKA                1.000 0.981          0.057 0.248 0.097 0.375  0.348 0.428   
ConcDiff           0.815 0.962          0.459 0.225 0.573 0.555  0.393 0.503   
DistCorr           1.000 0.994          0.049 0.165 0.084 0.379  0.282 0.313   
EOS                1.000 0.724          0.539 0.293 0.260 0.501  0.191 0.128   
GULP               0.986 0.724          0.537 0.282 0.261 0.499  0.189 0.508   
HardCorr           0.798 0.724          0.305 0.346 0.232 0.308  0.192 0.171   
IMD                0.609 0.918          0.121 0.078 0.041 0.309  0.379 0.055   
Jaccard            1.000 1.000          0.033 0.085 0.071 0.463  0.160 0.000   
LinReg             1.000 1.000          0.245 0.122 0.053 0.380  0.277 0.297   
MagDiff            0.550 0.535          0.511 0.255 0.585 0.417  0.523 0.331   
ShapeMet           1.000 1.000          0.097 0.213 0.085 0.373  0.279 0.202   
OrthProc           1.000 1.000          0.097 0.213 0.085 0.373  0.279 0.202   
PWCCA              0.990 0.724          0.439 0.265 0.289 0.481  0.265   NaN   
PermProc           0.649 0.433          0.359 0.434 0.328 0.435  0.424 0.281   
ProcDist           1.000 0.997          0.080 0.234 0.087 0.422  0.392 0.185   
RSA                0.966 0.978          0.086 0.283 0.216 0.301  0.187 0.030   
RSMDiff            1.000 1.000          0.091 0.303 0.243 0.357  0.405 0.251   
RankSim            1.000 1.000          0.047 0.131 0.105 0.419  0.170 0.000   
SVCCA              1.000 0.970          0.309 0.541 0.403 0.475  0.337 0.433   
2nd-Cos            1.000 1.

In [10]:
shortcut_test_df = get_agg_pivot_table("shortcut_test", ["cora", "flickr", "ogbn-arxiv"])
styled = pd.io.formats.style.Styler(
    shortcut_test_df,
    precision=2,
)

latex_str = styled.highlight_max(axis=0, props="textbf:--rwrap;").to_latex("graphs_results_shortcut_test.tex", **LATEX_FORMAT_DICT)

#### Augmentation Test Results

In [11]:
get_agg_pivot_table("augmentation_test", ["cora", "flickr", "ogbn-arxiv"])

Measure            AUPRC                                                  \
Dataset             Cora             Flickr             OGBN-Arxiv         
Model                GAT   GCN  SAGE    GAT   GCN  SAGE        GAT   GCN   
Similarity Measure                                                         
AlignCos           0.908 0.510 0.635  0.544 0.686 0.698      1.000 0.736   
CKA                0.906 0.697 0.731  0.508 0.572 0.747      0.987 0.997   
ConcDiff           0.623 0.174 0.504  0.534 0.412 0.348      0.508 0.426   
DistCorr           0.891 0.785 0.732  0.609 0.602 0.794      0.989 1.000   
EOS                0.342 0.535 0.618  0.459 0.679 0.529      1.000 0.819   
GULP               0.349 0.448 0.612  0.555 0.223 0.542      1.000 0.557   
HardCorr           0.513 0.512 0.633  0.571 0.712 0.716      0.536 0.466   
IMD                0.199 0.741 0.343  0.280 0.587 0.566      1.000 1.000   
Jaccard            0.953 0.781 0.807  0.580 0.743 0.880      1.000 1.000   
LinReg             0.872 0.943 0.850  0.338 0.302 0.805      0.809 0.724   
MagDiff            0.535 0.204 0.390  0.430 0.588 0.174      0.693 0.771   
ShapeMet           0.835 0.812 0.784  0.540 0.627 0.760      0.744 0.724   
OrthProc           0.835 0.812 0.784  0.540 0.627 0.760      0.744 0.724   
PWCCA              0.421 0.600 0.650  0.511   NaN 0.567      0.820   NaN   
PermProc           0.423 0.701 0.613  0.398 0.649 0.685      0.936 0.881   
ProcDist           0.749 0.764 0.693  0.401 0.696 0.814      1.000 1.000   
RSA                0.784 0.385 0.573  0.586 0.680 0.718      0.674 0.745   
RSMDiff            1.000 0.987 0.795  0.401 0.644 0.933      1.000 1.000   
RankSim            0.947 0.540 0.618  0.553 0.746 0.882      1.000 1.000   
SVCCA              0.339 0.267 0.294  0.495 0.643 0.673      0.820 0.889   
2nd-Cos            0.967 0.734 0.863  0.622 0.777 0.920      1.000 1.000   
SoftCorr           0.496 0.630 0.702  0.546 0.729 0.581      0.448 0.425   
UnifDiff           1.000 0.747 0.534  0.247 0.683 0.239      0.447 0.757   

Measure                  Violation Rate                                 \
Dataset                            Cora             Flickr               
Model               SAGE            GAT   GCN  SAGE    GAT   GCN  SAGE   
Similarity Measure                                                       
AlignCos           0.427          0.022 0.217 0.119  0.192 0.130 0.099   
CKA                1.000          0.035 0.103 0.081  0.264 0.173 0.088   
ConcDiff           0.533          0.133 0.508 0.219  0.258 0.221 0.319   
DistCorr           1.000          0.035 0.071 0.088  0.188 0.162 0.059   
EOS                0.488          0.248 0.143 0.104  0.297 0.125 0.203   
GULP               0.484          0.237 0.210 0.109  0.161 0.477 0.191   
HardCorr           0.506          0.195 0.151 0.105  0.149 0.106 0.084   
IMD                1.000          0.539 0.069 0.359  0.470 0.129 0.137   
Jaccard            0.991          0.016 0.096 0.054  0.159 0.115 0.027   
LinReg             0.724          0.048 0.017 0.037  0.309 0.473 0.067   
MagDiff            1.000          0.173 0.485 0.258  0.341 0.132 0.551   
ShapeMet           0.724          0.048 0.050 0.069  0.232 0.152 0.063   
OrthProc           0.724          0.048 0.050 0.069  0.232 0.152 0.063   
PWCCA              0.477          0.173 0.141 0.112  0.249   NaN 0.169   
PermProc           0.734          0.486 0.220 0.113  0.384 0.128 0.099   
ProcDist           1.000          0.103 0.115 0.095  0.374 0.113 0.051   
RSA                0.495          0.062 0.319 0.156  0.159 0.143 0.080   
RSMDiff            1.000          0.000 0.003 0.071  0.380 0.127 0.031   
RankSim            1.000          0.017 0.278 0.120  0.165 0.111 0.039   
SVCCA              0.813          0.421 0.457 0.449  0.316 0.162 0.136   
2nd-Cos            0.992          0.011 0.125 0.028  0.164 0.092 0.019   
SoftCorr           0.460          0.204 0.114 0.077  0.163 0.128 0.148   
UnifDiff           0.52

In [12]:
augment_test_df = get_agg_pivot_table("augmentation_test", ["cora", "flickr", "ogbn-arxiv"])
styled = pd.io.formats.style.Styler(
    augment_test_df,
    precision=2,
)

latex_str = styled.highlight_max(axis=0, props="textbf:--rwrap;").to_latex("graphs_results_augmentation_test.tex", **LATEX_FORMAT_DICT)

#### Layer Monotonicity Test Results

In [13]:
layer_test_df = get_agg_pivot_table("layer_test", ["cora", "flickr", "ogbn-arxiv"])
styled = pd.io.formats.style.Styler(
    layer_test_df,
    precision=2,
)

latex_str = styled.highlight_max(axis=0, props="textbf:--rwrap;").to_latex("graphs_results_layer_test.tex", **LATEX_FORMAT_DICT)

In [14]:
layer_test_df

Measure            Spearman Correlation                                        \
Dataset                            Cora                   Flickr                
Model                               GAT   GCN  SAGE  PGNN    GAT   GCN   SAGE   
Similarity Measure                                                              
AlignCos                          1.000 0.980 1.000 0.313  0.040 0.327  0.680   
CKA                               1.000 0.980 1.000 1.000  0.387 0.603  0.890   
ConcDiff                          0.403 0.850 0.250 0.117  0.243 0.383 -0.273   
DistCorr                          1.000 1.000 1.000 1.000  0.467 0.630  0.993   
EOS                               1.000 1.000 1.000 0.987  0.917 1.000  1.000   
GULP                              1.000 1.000 1.000 0.720  0.330 0.540  1.000   
HardCorr                          0.540 0.833 0.983 0.777  0.213 0.627  0.803   
IMD                               1.000 1.000 1.000 0.887  0.970 1.000  0.820   
Jaccard                           1.000 1.000 1.000 1.000  0.953 0.950  0.973   
LinReg                            1.000 1.000 1.000 1.000  0.030 0.450  1.000   
MagDiff                           0.893 0.550 0.490 0.910  0.383 0.583  0.500   
ShapeMet                          1.000 1.000 1.000 0.910  0.453 0.620  0.983   
OrthProc                          1.000 1.000 1.000 0.910  0.453 0.620  0.983   
PWCCA                             1.000 1.000 1.000 1.000  0.373   NaN  1.000   
PermProc                          1.000 0.917 1.000 1.000  0.630 0.603  0.680   
ProcDist                          1.000 0.993 1.000 0.910  0.617 0.660  1.000   
RSA                               1.000 0.837 0.910 0.607  0.437 0.580  0.987   
RSMDiff                           1.000 0.993 1.000 0.910  0.807 0.540  0.653   
RankSim                           1.000 1.000 1.000 1.000  0.987 0.950  0.967   
SVCCA                             0.613 0.410 0.777 1.000  0.467 0.533  0.440   
2nd-Cos                           1.000 1.000 1.000 0.900  0.517 0.810  0.957   
SoftCorr                          0.500 0.957 0.993 0.753  0.113 0.787  0.893   
UnifDiff                          0.590 0.667 0.667 0.597  0.820 0.820  0.890   

Measure                                     Violation Rate                    \
Dataset            OGBN-Arxiv                         Cora                     
Model                     GAT    GCN   SAGE            GAT   GCN  SAGE  PGNN   
Similarity Measure                                                             
AlignCos                0.837  0.843  0.933          0.000 0.030 0.000 0.120   
CKA                     0.960  0.850  0.457          0.000 0.020 0.000 0.000   
ConcDiff                0.397  0.733  0.390          0.380 0.150 0.480 0.490   
DistCorr                0.933  0.807  0.523          0.000 0.000 0.000 0.000   
EOS                     1.000  1.000  1.000          0.000 0.000 0.000 0.020   
GULP                    1.000  0.733  1.000          0.000 0.000 0.000 0.150   
HardCorr                1.000  0.833  0.620          0.240 0.090 0.010 0.180   
IMD                     1.000  0.550  1.000          0.000 0.000 0.000 0.050   
Jaccard                 1.000  0.973  0.987          0.000 0.000 0.000 0.000   
LinReg                  1.000  0.993  1.000          0.000 0.000 0.000 0.000   
MagDiff                 0.633  0.060  0.290          0.080 0.370 0.280 0.030   
ShapeMet                0.993  0.967  0.993          0.000 0.000 0.000 0.030   
OrthProc                0.993  0.967  0.993          0.000 0.000 0.000 0.030   
PWCCA                   1.000  1.000  1.000          0.000 0.000 0.000 0.020   
PermProc                1.000  0.243  0.883          0.000 0.090 0.000 0.000   
ProcDist                1.000  0.833  0.853          0.000 0.010 0.000 0.030   
RSA                     0.940  0.523  0.973          0.000 0.100 0.030 0.260   
RSMDiff                 1.000  0.850  0.933          0.000 0.010 0.000 0.030   
RankSim                 1.000  0.993  0.987         

### Results of Output Correlations

In [15]:
def get_output_correlation_table(datasets, corr_func="spearmanr", acc_test = False):
    
    full_csvs = os.listdir(EXPERIMENT_RESULTS_PATH)

    dfs = []
    for dataset in datasets:
        for f in full_csvs:
            if "output" in f and dataset in f:
                df = pd.read_csv(os.path.join(EXPERIMENT_RESULTS_PATH, f))
                df = df.loc[df.loc[:,"quality_measure"] == corr_func]
                data = df.loc[:, ["similarity_measure", "functional_similarity_measure", "corr", "architecture"]]
                data = data.rename(COLUMN_NAME_DICT, axis="columns")
                data["Dataset"] = dataset
                dfs.append(data.iloc[:])

    df_cc = pd.concat(dfs, axis=0)
    df_res = df_cc.pivot_table(index="Similarity Measure", columns=["Measure", "Dataset", "Model"], values="corr", aggfunc="mean")
    df_res = df_res.rename(MEASURE_NAME_DICT, axis="index")
    df_res = df_res.rename(PIVOT_COL_DICT, axis="columns")
    if acc_test: 
        return df_res.loc[:,"AbsoluteAccDiff"]
        
    return df_res.loc[:,["Disagreement", "JSD"]]

In [16]:
output_corr_df = get_output_correlation_table(["cora"])
styled = pd.io.formats.style.Styler(
    output_corr_df,
    precision=2,
)
styled.highlight_max(axis=0, props="textbf:--rwrap;").to_latex("cora_results_output_correlation_pgnn.tex", **LATEX_FORMAT_DICT)
output_corr_df

Measure            Disagreement                         JSD                \
Dataset                    Cora                        Cora                 
Model                       GAT    GCN   SAGE   PGNN    GAT    GCN   SAGE   
Similarity Measure                                                          
AlignCos                 -0.051 -0.084  0.271  0.206  0.174  0.017  0.384   
CKA                       0.445  0.363  0.001  0.137  0.535  0.435 -0.031   
ConcDiff                  0.030 -0.038 -0.097 -0.066  0.014  0.032  0.041   
DistCorr                  0.456  0.529 -0.078  0.284  0.601  0.597  0.053   
EOS                      -0.088  0.127  0.023  0.294 -0.036  0.168  0.224   
GULP                     -0.129 -0.165 -0.090  0.334 -0.042 -0.096  0.182   
HardCorr                  0.261  0.454  0.104  0.450  0.516  0.479  0.162   
IMD                      -0.150  0.180 -0.103 -0.135 -0.086  0.213 -0.404   
Jaccard                   0.120  0.436  0.333  0.296  0.379  0.441  0.453   
LinReg                   -0.062  0.574  0.090  0.228  0.190  0.580  0.331   
MagDiff                  -0.069 -0.073 -0.253  0.250 -0.145  0.004  0.063   
ShapeMet                  0.053  0.616  0.156  0.312  0.282  0.695  0.272   
OrthProc                  0.053  0.616  0.156  0.312  0.282  0.695  0.272   
PWCCA                    -0.138  0.081  0.360  0.212  0.015  0.080  0.267   
PermProc                 -0.138  0.235  0.328  0.422  0.133  0.289  0.096   
ProcDist                 -0.175  0.450  0.237  0.427  0.046  0.516  0.085   
RSA                      -0.018  0.055  0.328  0.479  0.200  0.117  0.133   
RSMDiff                  -0.150  0.050  0.137  0.345 -0.072  0.115 -0.011   
RankSim                   0.442  0.343  0.227  0.303  0.505  0.363  0.561   
SVCCA                     0.166 -0.057  0.045  0.004  0.261 -0.085  0.018   
2nd-Cos                   0.299  0.399  0.252  0.153  0.529  0.429  0.536   
SoftCorr                  0.069  0.485  0.057  0.405  0.215  0.534 -0.013   
UnifDiff                 -0.146  0.067  0.161  0.276 -0.114  0.133  0.270   

Measure                    
Dataset                    
Model                PGNN  
Similarity Measure         
AlignCos           -0.143  
CKA                -0.327  
ConcDiff            0.205  
DistCorr           -0.362  
EOS                -0.225  
GULP                0.100  
HardCorr           -0.174  
IMD                 0.104  
Jaccard             0.051  
LinReg             -0.267  
MagDiff             0.101  
ShapeMet           -0.251  
OrthProc           -0.251  
PWCCA              -0.270  
PermProc            0.006  
ProcDist            0.012  
RSA                 0.106  
RSMDiff             0.006  
RankSim            -0.042  
SVCCA              -0.027  
2nd-Cos            -0.240  
SoftCorr           -0.168  
UnifDiff           -0.220

In [17]:
output_corr_df = get_output_correlation_table(["cora", "flickr", "ogbn-arxiv"])
styled = pd.io.formats.style.Styler(
    output_corr_df,
    precision=2,
)
styled.highlight_max(axis=0, props="textbf:--rwrap;").to_latex("graphs_results_output_correlation.tex", **LATEX_FORMAT_DICT)
output_corr_df

Measure            Disagreement                                            \
Dataset                    Cora                      Flickr                 
Model                       GAT    GCN   SAGE   PGNN    GAT    GCN   SAGE   
Similarity Measure                                                          
AlignCos                 -0.051 -0.084  0.271  0.206 -0.083  0.147  0.374   
CKA                       0.445  0.363  0.001  0.137  0.061 -0.211  0.534   
ConcDiff                  0.030 -0.038 -0.097 -0.066  0.029 -0.214 -0.043   
DistCorr                  0.456  0.529 -0.078  0.284 -0.031  0.172  0.395   
EOS                      -0.088  0.127  0.023  0.294  0.231  0.025  0.334   
GULP                     -0.129 -0.165 -0.090  0.334 -0.006  0.029  0.334   
HardCorr                  0.261  0.454  0.104  0.450 -0.093  0.395  0.458   
IMD                      -0.150  0.180 -0.103 -0.135 -0.023 -0.098  0.330   
Jaccard                   0.120  0.436  0.333  0.296 -0.004  0.037  0.419   
LinReg                   -0.062  0.574  0.090  0.228  0.063  0.119  0.457   
MagDiff                  -0.069 -0.073 -0.253  0.250 -0.198  0.056  0.065   
ShapeMet                  0.053  0.616  0.156  0.312  0.034  0.193  0.573   
OrthProc                  0.053  0.616  0.156  0.312  0.034  0.192  0.573   
PWCCA                    -0.138  0.081  0.360  0.212  0.159    NaN  0.318   
PermProc                 -0.138  0.235  0.328  0.422 -0.272  0.178 -0.097   
ProcDist                 -0.175  0.450  0.237  0.427 -0.265 -0.134 -0.148   
RSA                      -0.018  0.055  0.328  0.479  0.063  0.348  0.586   
RSMDiff                  -0.150  0.050  0.137  0.345 -0.261 -0.297 -0.003   
RankSim                   0.442  0.343  0.227  0.303  0.047  0.055  0.237   
SVCCA                     0.166 -0.057  0.045  0.004  0.028 -0.269  0.214   
2nd-Cos                   0.299  0.399  0.252  0.153 -0.103  0.301  0.143   
SoftCorr                  0.069  0.485  0.057  0.405  0.017  0.390  0.486   
UnifDiff                 -0.146  0.067  0.161  0.276  0.182 -0.340  0.038   

Measure                                        JSD                       \
Dataset            OGBN-Arxiv                 Cora                        
Model                     GAT    GCN   SAGE    GAT    GCN   SAGE   PGNN   
Similarity Measure                                                        
AlignCos                0.166 -0.098  0.001  0.174  0.017  0.384 -0.143   
CKA                     0.227  0.029 -0.041  0.535  0.435 -0.031 -0.327   
ConcDiff               -0.160 -0.254  0.072  0.014  0.032  0.041  0.205   
DistCorr                0.204  0.110  0.081  0.601  0.597  0.053 -0.362   
EOS                     0.119 -0.255 -0.023 -0.036  0.168  0.224 -0.225   
GULP                    0.099  0.154 -0.037 -0.042 -0.096  0.182  0.100   
HardCorr                0.239  0.020 -0.241  0.516  0.479  0.162 -0.174   
IMD                     0.037 -0.125  0.012 -0.086  0.213 -0.404  0.104   
Jaccard                 0.219  0.142  0.012  0.379  0.441  0.453  0.051   
LinReg                  0.221 -0.099 -0.188  0.190  0.580  0.331 -0.267   
MagDiff                 0.216 -0.252 -0.195 -0.145  0.004  0.063  0.101   
ShapeMet                0.268  0.018 -0.165  0.282  0.695  0.272 -0.251   
OrthProc                0.268  0.018 -0.165  0.282  0.695  0.272 -0.251   
PWCCA                   0.033 -0.243 -0.079  0.015  0.080  0.267 -0.270   
PermProc                0.221  0.250 -0.412  0.133  0.289  0.096  0.006   
ProcDist                0.303  0.257 -0.382  0.046  0.516  0.085  0.012   
RSA                     0.454  0.170  0.155  0.200  0.117  0.133  0.106   
RSMDiff                 0.356 -0.141  0.090 -0.072  0.115 -0.011  0.006   
RankSim                 0.096  0.027  0.274  0.505  0.363  0.561 -0.042   
SVCCA                  -0.146  0.085  0.113  0.261 -0.085  0.018 -0.027   
2nd-Cos                 0.064  0.042  0.265  0.529  0.429  0.536 -0.240   
SoftCorr                0.353  0.034 -0.260  

In [18]:
acc_corr_df = get_output_correlation_table(["cora", "flickr", "ogbn-arxiv"], acc_test=True)
styled = pd.io.formats.style.Styler(
    acc_corr_df,
    precision=2,
)
styled.highlight_max(axis=0, props="textbf:--rwrap;").to_latex("graphs_results_accuracy_correlation.tex", **LATEX_FORMAT_DICT)
acc_corr_df

Dataset              Cora                      Flickr                \
Model                 GAT    GCN   SAGE   PGNN    GAT    GCN   SAGE   
Similarity Measure                                                    
AlignCos           -0.289 -0.332  0.132  0.050 -0.067  0.354  0.237   
CKA                 0.023  0.163 -0.172 -0.009 -0.160  0.025  0.267   
ConcDiff           -0.197  0.154 -0.252  0.354 -0.071 -0.079 -0.287   
DistCorr            0.026  0.008 -0.175  0.153 -0.193  0.413  0.416   
EOS                -0.049 -0.236  0.080  0.272  0.290  0.149 -0.265   
GULP               -0.121 -0.435  0.081  0.276 -0.273  0.036 -0.271   
HardCorr           -0.143  0.106 -0.109  0.322  0.059  0.306  0.352   
IMD                -0.023 -0.114 -0.005 -0.157 -0.089 -0.095  0.366   
Jaccard            -0.160  0.023 -0.111  0.079 -0.180  0.317  0.281   
LinReg             -0.109  0.058 -0.207  0.061 -0.180 -0.036  0.174   
MagDiff            -0.192  0.077 -0.133  0.357  0.145  0.023 -0.173   
ShapeMet           -0.118  0.147 -0.292  0.202 -0.147  0.395  0.280   
OrthProc           -0.118  0.147 -0.292  0.202 -0.147  0.394  0.280   
PWCCA              -0.261 -0.050  0.068  0.085 -0.163    NaN -0.051   
PermProc           -0.261 -0.118  0.181  0.475  0.150  0.205 -0.195   
ProcDist           -0.192  0.083  0.006  0.475  0.113  0.016 -0.065   
RSA                -0.279 -0.271  0.043  0.375 -0.082  0.531  0.316   
RSMDiff            -0.141 -0.189  0.074  0.448  0.126 -0.184 -0.161   
RankSim             0.321 -0.004 -0.103  0.068 -0.198  0.346  0.312   
SVCCA              -0.186 -0.015 -0.126 -0.015 -0.176  0.007  0.015   
2nd-Cos            -0.109 -0.100  0.043  0.150  0.009  0.544 -0.187   
SoftCorr            0.026  0.183 -0.053  0.346 -0.075  0.301  0.331   
UnifDiff           -0.088 -0.103 -0.037 -0.012 -0.183 -0.175  0.034   

Dataset            OGBN-Arxiv                
Model                     GAT    GCN   SAGE  
Similarity Measure                           
AlignCos               -0.172 -0.081  0.168  
CKA                    -0.052 -0.167  0.106  
ConcDiff               -0.117 -0.073 -0.133  
DistCorr               -0.060 -0.098  0.146  
EOS                    -0.324 -0.213  0.054  
GULP                   -0.345 -0.084  0.061  
HardCorr                0.040  0.366  0.015  
IMD                    -0.148 -0.207 -0.020  
Jaccard                -0.145 -0.321 -0.130  
LinReg                 -0.188  0.070 -0.011  
MagDiff                 0.109 -0.180 -0.198  
ShapeMet               -0.086 -0.035  0.090  
OrthProc               -0.086 -0.035  0.090  
PWCCA                  -0.296 -0.091  0.059  
PermProc                0.426 -0.094  0.032  
ProcDist                0.430 -0.174  0.069  
RSA                     0.317 -0.071  0.248  
RSMDiff                 0.019 -0.046 -0.194  
RankSim                -0.090 -0.277  0.049  
SVCCA                  -0.100 -0.268  0.095  
2nd-Cos                -0.189 -0.471  0.224  
SoftCorr                0.122  0.354  0.121  
UnifDiff               -0.250 -0.187 -0.203